In [1]:
import decimal
import datetime
import json
import pymysql

In [2]:
MYSQL_HOST_IP = "127.0.0.1"
MYSQL_USERNAME = 'root'
MYSQL_PASSWORD = 'TestSQLPassword789'

In [3]:
class SQLJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, decimal.Decimal):
            return float(obj)
        if isinstance(obj, (datetime.datetime, datetime.date)):
            return obj.isoformat()
        return super(SQLJSONEncoder, self).default(obj)

def execute_sql_cmd(sql_query):
    """
    Input:
        sql_query: SQL query, for example, SELECT a, b FROM table WHERE c=c1
    """
    # Connect to the database
    conn = pymysql.connect(
        host=MYSQL_HOST_IP,
        user=MYSQL_USERNAME,
        password=MYSQL_PASSWORD,
        cursorclass=pymysql.cursors.DictCursor
    )
    try:
        with conn.cursor() as cursor:
            cursor.execute(sql_query)
            result = cursor.fetchall()
            if result is not None:
                print(json.dumps(result, cls=SQLJSONEncoder, indent=4))
        conn.commit()
    except Exception as e:
        print(f"SQL Command: [{sql_query}]; Error: {e}")
    finally:
        conn.close()

## DDL
### DDL: Prepare database and tables

In [4]:
ddl_cmd_list = [
    "CREATE DATABASE IF NOT EXISTS sql_exercises;",
    "CREATE TABLE IF NOT EXISTS sql_exercises.Student (`S#` VARCHAR(10), `Sname` VARCHAR(50), `Sage` DATE, `Ssex` VARCHAR(10), PRIMARY KEY (`S#`));",
    "CREATE TABLE IF NOT EXISTS sql_exercises.Course(`C#` VARCHAR(10), `Cname` VARCHAR(50), `T#` VARCHAR(10), PRIMARY KEY (`C#`));",    
    "CREATE TABLE IF NOT EXISTS sql_exercises.Teacher(`T#` VARCHAR(10), `Tname` VARCHAR(50), PRIMARY KEY (`T#`));",
    "CREATE TABLE IF NOT EXISTS sql_exercises.SC(`S#` VARCHAR(10), `C#` VARCHAR(10), score DECIMAL(18,1), PRIMARY KEY (`S#`, `C#`));",
]

In [5]:
[execute_sql_cmd(ddl_cmd) for ddl_cmd in ddl_cmd_list]

[]
[]
[]
[]
[]


/home/huawei/.local/lib/python3.6/site-packages/pymysql/cursors.py:329: Warning: (1007, "Can't create database 'sql_exercises'; database exists")
  self._do_get_result()


[None, None, None, None, None]

### DDL: Remove tables or database in case of schema change 

In [6]:
#ddl_cmd_list = [
#    "DROP TABLE IF EXISTS sql_exercises.Student, sql_exercises.Course, sql_exercises.Teacher, sql_exercises.SC;",
#    #"CREATE DATABASE IF EXISTS sql_exercises;",
#]

In [7]:
#[execute_sql_cmd(ddl_cmd) for ddl_cmd in ddl_cmd_list]

### DML: Insert entries

In [8]:
dml_cmd_list = [
    # data for student table
    "INSERT INTO sql_exercises.Student values('01', 'Rosie Nicholson', '1991-11-01', 'F');",
    "INSERT INTO sql_exercises.Student values('02', 'Kyle Cooper', '1993-02-28', 'M');",
    "INSERT INTO sql_exercises.Student values('03', 'Will Joyner', '1991-05-20', 'M');",
    "INSERT INTO sql_exercises.Student values('04', 'Warren Mccarthy', '1992-08-28', 'M');",
    "INSERT INTO sql_exercises.Student values('05', 'Ben Barnes', '1993-12-01', 'M');",
    "INSERT INTO sql_exercises.Student values('06', 'Deborah Long', '1992-03-01', 'F');",
    "INSERT INTO sql_exercises.Student values('07', 'Jackson Burke', '1992-04-11', 'M');",
    "INSERT INTO sql_exercises.Student values('08', 'Xander Randall', '1990-11-13', 'M');",
    # data for Course table
    "INSERT INTO sql_exercises.Course values('01', 'General Physics', '02');",
    "INSERT INTO sql_exercises.Course values('02', 'Condensed Matter Physics', '01');",
    "INSERT INTO sql_exercises.Course values('03', 'Particle Physics', '03');",
    # data for Teacher table,
    "INSERT INTO sql_exercises.Teacher values('01', 'Lev Landau');",
    "INSERT INTO sql_exercises.Teacher values('02', 'Richard Feynman');",
    "INSERT INTO sql_exercises.Teacher values('03', 'Nishina, Yoshio');",
    # data for SC table,
    "INSERT INTO sql_exercises.SC values('01', '01', 80);",
    "INSERT INTO sql_exercises.SC values('01', '02', 90);",
    "INSERT INTO sql_exercises.SC values('01', '03', 99);",
    "INSERT INTO sql_exercises.SC values('02', '01', 70);",
    "INSERT INTO sql_exercises.SC values('02', '02', 60);",
    "INSERT INTO sql_exercises.SC values('02', '03', 80);",
    "INSERT INTO sql_exercises.SC values('03', '01', 80);",
    "INSERT INTO sql_exercises.SC values('03', '02', 80);",
    "INSERT INTO sql_exercises.SC values('03', '03', 80);",
    "INSERT INTO sql_exercises.SC values('04', '01', 50);",
    "INSERT INTO sql_exercises.SC values('04', '02', 30);",
    "INSERT INTO sql_exercises.SC values('04', '03', 20);",
    "INSERT INTO sql_exercises.SC values('05', '01', 76);",
    "INSERT INTO sql_exercises.SC values('05', '02', 87);",
    "INSERT INTO sql_exercises.SC values('06', '01', 31);",
    "INSERT INTO sql_exercises.SC values('06', '03', 34);",
    "INSERT INTO sql_exercises.SC values('07', '02', 89);",
    "INSERT INTO sql_exercises.SC values('07', '03', 98);",
]

In [9]:
[execute_sql_cmd(dml_cmd) for dml_cmd in dml_cmd_list]

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### DQL: Select data

#### Q1.1: Get the S#, C#, and corresponding courses' scores if the student 01's course score is higher than 02's.

In [10]:
dql_cmd = (
"SELECT A.*, B.`C#`, B.score FROM \
(SELECT * FROM sql_exercises.SC WHERE `C#`='01') AS A INNER JOIN \
(SELECT * FROM sql_exercises.SC WHERE `C#`='02') AS B ON A.`S#`=B.`S#` \
WHERE A.score > B.score;"
)

In [11]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "02",
        "C#": "01",
        "score": 70.0,
        "B.C#": "02",
        "B.score": 60.0
    },
    {
        "S#": "04",
        "C#": "01",
        "score": 50.0,
        "B.C#": "02",
        "B.score": 30.0
    }
]


### Q1.2: Get the S#, C# and score if one student choose both course 01 and course 02

In [12]:
dql_cmd = (
"SELECT * FROM \
(SELECT * FROM sql_exercises.SC WHERE `C#`='01') AS A INNER JOIN \
(SELECT * FROM sql_exercises.SC WHERE `C#`='02') AS B ON A.`S#`=B.`S#`;"
)

In [13]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "C#": "01",
        "score": 80.0,
        "B.S#": "01",
        "B.C#": "02",
        "B.score": 90.0
    },
    {
        "S#": "02",
        "C#": "01",
        "score": 70.0,
        "B.S#": "02",
        "B.C#": "02",
        "B.score": 60.0
    },
    {
        "S#": "03",
        "C#": "01",
        "score": 80.0,
        "B.S#": "03",
        "B.C#": "02",
        "B.score": 80.0
    },
    {
        "S#": "04",
        "C#": "01",
        "score": 50.0,
        "B.S#": "04",
        "B.C#": "02",
        "B.score": 30.0
    },
    {
        "S#": "05",
        "C#": "01",
        "score": 76.0,
        "B.S#": "05",
        "B.C#": "02",
        "B.score": 87.0
    }
]


### Q1.3: Get the S#, C# and score if one student choose course 02 but not course 01

In [14]:
dql_cmd = (
"SELECT * FROM sql_exercises.SC WHERE `C#`='02' AND `S#` NOT IN \
(SELECT `S#` FROM sql_exercises.SC WHERE `C#`='01');"
)

In [15]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "07",
        "C#": "02",
        "score": 89.0
    }
]


### Q2.1: Get the S#, Sname and his/her average course score if one student's average course score is greater than 60

In [16]:
dql_cmd = (
" SELECT A.`S#`, B.Sname, A.avgs FROM \
(SELECT `S#`, AVG(score) as avgs FROM sql_exercises.SC GROUP BY `S#`) AS A INNER JOIN \
sql_exercises.Student AS B ON A.`S#`=B.`S#` WHERE A.avgs>60;"
)

In [17]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "avgs": 89.66667
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "avgs": 70.0
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "avgs": 80.0
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "avgs": 81.5
    },
    {
        "S#": "07",
        "Sname": "Jackson Burke",
        "avgs": 93.5
    }
]


### Q3.1: Get all information in Student table that S# existed in SC table 

In [18]:
dql_cmd = (
"SELECT * FROM sql_exercises.Student WHERE `S#` IN \
(SELECT `S#` FROM sql_exercises.SC)"
)

In [19]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F"
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M"
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M"
    },
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M"
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M"
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F"
    },
    {
        "S#": "07",
        "Sname": "Jackson Burke",
        "Sage": "1992-04-11",
        "Ssex": "M"
    }
]


### Q4.1: Get all students information with number of courses and total course scores (Null if one student do not choose any courses)

In [20]:
dql_cmd = (
"SELECT B.*, A.CourseNumber, A.TotalScore FROM \
(SELECT `S#`, COUNT(`C#`) as CourseNumber, SUM(`score`) as TotalScore FROM sql_exercises.SC GROUP BY `S#`) AS A RIGHT JOIN \
sql_exercises.Student AS B ON A.`S#`=B.`S#`;"
)

In [21]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F",
        "CourseNumber": 3,
        "TotalScore": 269.0
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 210.0
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 240.0
    },
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 100.0
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M",
        "CourseNumber": 2,
        "TotalScore": 163.0
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F",
        "CourseNumber": 2,

### Q4.2: Get all students information with number of courses and total course scores who choose at least one course

In [22]:
dql_cmd = (
"SELECT B.*, A.CourseNumber, A.TotalScore FROM \
(SELECT `S#`, COUNT(`C#`) as CourseNumber, SUM(`score`) as TotalScore FROM sql_exercises.SC GROUP BY `S#`) AS A LEFT JOIN \
sql_exercises.Student AS B ON A.`S#`=B.`S#`;"
)

In [23]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F",
        "CourseNumber": 3,
        "TotalScore": 269.0
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 210.0
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 240.0
    },
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M",
        "CourseNumber": 3,
        "TotalScore": 100.0
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M",
        "CourseNumber": 2,
        "TotalScore": 163.0
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F",
        "CourseNumber": 2,

### Q5.1: Get Teacher information who's name contain 'Landau'

In [24]:
dql_cmd = (
"SELECT * FROM sql_exercises.Teacher WHERE Tname LIKE '%Landau%';"
)

In [25]:
execute_sql_cmd(dql_cmd)

[
    {
        "T#": "01",
        "Tname": "Lev Landau"
    }
]


### Q6.1: Get the student information who has take course from Teacher 'Richard Feynman'

In [26]:
dql_cmd = (
"SELECT * FROM sql_exercises.Student WHERE `S#` IN \
(SELECT DISTINCT `S#` FROM sql_exercises.SC WHERE `C#` IN \
(SELECT `C#` FROM sql_exercises.Course WHERE `T#` IN \
(SELECT `T#` FROM sql_exercises.Teacher WHERE Tname='Richard Feynman')));"
)

In [27]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F"
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M"
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M"
    },
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M"
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M"
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F"
    }
]


### Q7.1: Get student information who not take all three courses

In [28]:
dql_cmd = (
"SELECT * FROM sql_exercises.Student WHERE `S#` IN \
(SELECT `S#` FROM sql_exercises.SC GROUP BY `S#` HAVING COUNT(`C#`)<3);"
)

In [29]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M"
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F"
    },
    {
        "S#": "07",
        "Sname": "Jackson Burke",
        "Sage": "1992-04-11",
        "Ssex": "M"
    }
]


### Q8.1: Get student information who take at least one course that same as student with S# 01

In [30]:
dql_cmd = (
"SELECT * FROM sql_exercises.Student WHERE `S#` IN \
(SELECT DISTINCT `S#` FROM sql_exercises.SC WHERE `C#` IN \
(SELECT `C#` FROM sql_exercises.SC WHERE `S#`='01'));"
)

In [31]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F"
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M"
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M"
    },
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M"
    },
    {
        "S#": "05",
        "Sname": "Ben Barnes",
        "Sage": "1993-12-01",
        "Ssex": "M"
    },
    {
        "S#": "06",
        "Sname": "Deborah Long",
        "Sage": "1992-03-01",
        "Ssex": "F"
    },
    {
        "S#": "07",
        "Sname": "Jackson Burke",
        "Sage": "1992-04-11",
        "Ssex": "M"
    }
]


### Q9.1: Get the student information who take completely same courses as student with S# 01

In [32]:
dql_cmd = (
"SELECT * FROM sql_exercises.Student WHERE `S#` IN \
(SELECT A.`S#` FROM \
(SELECT `S#`, GROUP_CONCAT(`C#` ORDER BY `C#` ASC) AS OCs FROM sql_exercises.SC GROUP BY `S#`) AS A INNER JOIN \
(SELECT `S#`, GROUP_CONCAT(`C#` ORDER BY `C#` ASC) AS OCs FROM sql_exercises.SC GROUP BY `S#` HAVING `S#`='01') AS B \
ON A.OCs=B.OCs);"
)

In [33]:
execute_sql_cmd(dql_cmd)

[
    {
        "S#": "04",
        "Sname": "Warren Mccarthy",
        "Sage": "1992-08-28",
        "Ssex": "M"
    },
    {
        "S#": "03",
        "Sname": "Will Joyner",
        "Sage": "1991-05-20",
        "Ssex": "M"
    },
    {
        "S#": "02",
        "Sname": "Kyle Cooper",
        "Sage": "1993-02-28",
        "Ssex": "M"
    },
    {
        "S#": "01",
        "Sname": "Rosie Nicholson",
        "Sage": "1991-11-01",
        "Ssex": "F"
    }
]


### Q10.1: Get the student name (Sname) who have not take course from teacher 'Lev Landau'

In [34]:
dql_cmd = (
"SELECT Sname FROM sql_exercises.Student WHERE `S#` NOT IN \
(SELECT `S#` FROM sql_exercises.SC WHERE `C#` IN \
(SELECT `C#` FROM sql_exercises.Course WHERE `T#` IN \
(SELECT `T#` FROM sql_exercises.Teacher WHERE `Tname`='Lev Landau')));"
)

In [35]:
execute_sql_cmd(dql_cmd)

[
    {
        "Sname": "Deborah Long"
    },
    {
        "Sname": "Xander Randall"
    }
]


### End of file
More information: https://blog.csdn.net/flycat296/article/details/63681089